<a href="https://colab.research.google.com/github/humanet1603/week1/blob/main/%EC%9C%A0%ED%8A%9C%EB%B8%8C%EC%9E%90%EB%A7%89%EB%B0%9B%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-api-python-client pytz


In [ ]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pytz

# 사용자로부터 API 키 수동 입력
API_KEY = input("API Key를 입력하세요: ")

# 사용자로부터 유튜브 채널 사용자명 입력받기

raw_input = input("채널 URL 또는 사용자명을 입력하세요 (예: @kosungkooktv 또는 kosungkooktv): ")
CHANNEL_USERNAME = raw_input.replace("@", "").strip()

# 채널 ID 조회
search_response = youtube.search().list(
    q=CHANNEL_USERNAME,
    type="channel",
    part="snippet",
    maxResults=1
).execute()

if not search_response['items']:
    print("채널을 찾을 수 없습니다.")
    exit()

channel_id = search_response['items'][0]['snippet']['channelId']


# YouTube API 클라이언트 생성
youtube = build('youtube', 'v3', developerKey=API_KEY)

# 채널 ID 조회 (username 기반)
search_response = youtube.search().list(
    q=CHANNEL_USERNAME,
    type="channel",
    part="snippet",
    maxResults=1
).execute()

if not search_response['items']:
    print("채널을 찾을 수 없습니다.")
    exit()

channel_id = search_response['items'][0]['snippet']['channelId']

# 채널의 업로드 playlistId 확인
channel_response = youtube.channels().list(
    part="contentDetails",
    id=channel_id
).execute()

uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

from datetime import datetime

# 사용자로부터 시작일과 마침일 입력받기
start_str = input("조회 시작 날짜를 입력하세요 (예: 2025-04-01): ")
end_str = input("조회 마침 날짜를 입력하세요 (예: 2025-04-05): ")

# 문자열을 datetime 객체로 변환
try:
    start_date = datetime.strptime(start_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_str, "%Y-%m-%d")

    # 마침일에 하루를 더해서 '포함' 처리
    end_date = end_date.replace(hour=23, minute=59, second=59)

    print(f"\n[조회 기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}]")
except ValueError:
    print("날짜 형식이 올바르지 않습니다. 'YYYY-MM-DD' 형식으로 입력해 주세요.")
    exit()

# 영상 수집
next_page_token = None
videos = []

next_page_token = None
videos = []

while True:
    playlist_response = youtube.playlistItems().list(
        part="snippet",
        playlistId=uploads_playlist_id,
        maxResults=50,
        pageToken=next_page_token
    ).execute()

    for item in playlist_response['items']:
        snippet = item['snippet']
        published_at = datetime.strptime(snippet['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')

        # 사용자 입력 기간 내 동영상만 수집
        if not (start_date <= published_at <= end_date):
            continue

        videos.append({
            "title": snippet['title'],
            "videoId": snippet['resourceId']['videoId'],
            "publishedAt": published_at,
            "description": snippet.get('description', ''),
            "thumbnail": snippet['thumbnails']['high']['url']
        })

    next_page_token = playlist_response.get('nextPageToken')
    if not next_page_token:
        break

# 결과 출력
print(f"\n최근 2일간 업로드된 영상 수: {len(videos)}개\n")
for v in videos:
    print(f"제목: {v['title']}")
    print(f"영상 링크: https://www.youtube.com/watch?v={v['videoId']}")
    print(f"업로드 시간(UTC): {v['publishedAt']}")
    print(f"썸네일: {v['thumbnail']}")
    print(f"설명: {v['description'][:100]}{'...' if len(v['description']) > 100 else ''}")
    print("-" * 80)


In [ ]:
!pip install youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

# videos 리스트는 앞서 수집한 동영상 메타데이터 정보라고 가정
for v in videos:
    video_id = v['videoId']
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
        full_text = "\n".join([entry['text'] for entry in transcript])
        filename = f"{video_id}_{v['title'][:30]}.txt"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(full_text)
        print(f"{filename} 저장 완료")
    except Exception as e:
        print(f"{v['title']} 자막 없음 또는 오류: {e}")


In [ ]:
import os

# 현재 작업 디렉토리 설정 (필요시 경로 수정 가능)
directory = os.getcwd()

# 병합 대상 파일 확장자
target_extension = ".txt"

# 출력 파일 이름
output_filename = "merged_transcripts.txt"

# 모든 txt 파일 이름 수집 (output 파일은 제외)
txt_files = [f for f in os.listdir(directory) if f.endswith(target_extension) and f != output_filename]

# 파일 이름 기준으로 정렬 (선택 사항)
txt_files.sort()

# 파일 병합
with open(output_filename, "w", encoding="utf-8") as outfile:
    for filename in txt_files:
        outfile.write(f"\n\n===== {filename} =====\n\n")  # 구분용 제목
        with open(os.path.join(directory, filename), "r", encoding="utf-8") as infile:
            outfile.write(infile.read())

print(f"\n{len(txt_files)}개의 파일이 '{output_filename}'로 병합되었습니다.")
